# Claude에서 "JSON 모드" 사용을 위한 프롬프트 작성법

Claude에는 제한된 샘플링을 사용하는 공식적인 "JSON 모드"는 없습니다. 하지만 걱정하지 마십시오. Claude로부터 안정적으로 JSON 형식의 응답을 받을 수 있습니다! 이 가이드에서 그 방법을 안내해 드리겠습니다.

먼저 Claude의 일반적인 응답 방식을 살펴보겠습니다.

In [ ]:
%pip install anthropic

In [1]:
from anthropic import Anthropic
import json
import re
from pprint import pprint

In [3]:
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [4]:
message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[
        {
            "role": "user", 
            "content": "유명 운동선수들의 이름과 해당 스포츠 종목을 담은 JSON 딕셔너리를 만들어 주세요."
        },
    ]
).content[0].text
print(message)

요청하신 유명 운동선수들의 이름과 각 스포츠 종목이 포함된 JSON 사전입니다:

```json
  "athletes": [
    {
      "name": "Usain Bolt",
      "sport": "Track and Field"
    },
    {
      "name": "Michael Phelps",
      "sport": "Swimming"
    },
    {
      "name": "Serena Williams",
      "sport": "Tennis"
    },
    {
      "name": "LeBron James",
      "sport": "Basketball"
    },
    {
      "name": "Lionel Messi",
      "sport": "Soccer"
    },
    {
      "name": "Simone Biles",
      "sport": "Gymnastics"
    },
    {
      "name": "Tom Brady",
      "sport": "American Football"
    },
    {
      "name": "Muhammad Ali",
      "sport": "Boxing"
    },
    {
      "name": "Nadia Comaneci",
      "sport": "Gymnastics"
    },
    {
      "name": "Michael Jordan",
      "sport": "Basketball"
    },
    {
      "name": "Pelé",
      "sport": "Soccer"
    },
    {
      "name": "Roger Federer",
      "sport": "Tennis"
    }
  ]
}
```

Claude는 지시사항을 잘 따랐고, 멋진 딕셔너리를 출력했습니다. 이제 코드를 사용하여 이 JSON을 추출할 수 있습니다:

In [8]:
def extract_json(response):
    json_start = response.index("{")
    json_end = response.rfind("}")
    return json.loads(response[json_start:json_end + 1])
extract_json(message)

{'athletes': [{'name': 'Usain Bolt', 'sport': 'Track and Field'},
  {'name': 'Michael Phelps', 'sport': 'Swimming'},
  {'name': 'Serena Williams', 'sport': 'Tennis'},
  {'name': 'LeBron James', 'sport': 'Basketball'},
  {'name': 'Lionel Messi', 'sport': 'Soccer'},
  {'name': 'Simone Biles', 'sport': 'Gymnastics'},
  {'name': 'Tom Brady', 'sport': 'American Football'},
  {'name': 'Muhammad Ali', 'sport': 'Boxing'},
  {'name': 'Nadia Comaneci', 'sport': 'Gymnastics'},
  {'name': 'Michael Jordan', 'sport': 'Basketball'},
  {'name': 'Pelé', 'sport': 'Soccer'},
  {'name': 'Roger Federer', 'sport': 'Tennis'}]}

하지만 Claude가 서론 없이 바로 JSON을 출력하도록 하려면 어떻게 해야 할까요? 한 가지 간단한 방법은 Claude의 응답 시작 부분에 "{" 문자를 미리 채워 넣어주는 것입니다.

In [9]:
message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[
        {
            "role": "user", 
            "content": "유명 운동선수들의 이름과 해당 스포츠 종목을 담은 JSON 딕셔너리를 만들어 주세요."
        },
        {
            "role": "assistant",
            "content": "요청하신 JSON은 다음과 같습니다:\n{"
        }
    ]
).content[0].text
print(message)


   "athletes":[
      {
         "name":"Michael Jordan",
         "sport":"Basketball"
      },
      {
         "name":"Babe Ruth",
         "sport":"Baseball"
      },
      {
         "name":"Muhammad Ali",
         "sport":"Boxing"
      },
      {
         "name":"Serena Williams",
         "sport":"Tennis"
      },
      {
         "name":"Wayne Gretzky",
         "sport":"Hockey"
      },
      {
         "name":"Michael Phelps",
         "sport":"Swimming"
      },
      {
         "name":"Usain Bolt",
         "sport":"Track and Field"
      },
      {
         "name":"Mia Hamm",
         "sport":"Soccer"
      },
      {
         "name":"Michael Schumacher",
         "sport":"Formula 1 Racing"
      },
      {
         "name":"Simone Biles",
         "sport":"Gymnastics"
      }
   ]
}


이제 우리가 미리 채웠던 "{" 문자를 다시 추가해주기만 하면 JSON을 성공적으로 추출할 수 있습니다.

In [11]:
output_json = json.loads("{" + message[:message.rfind("}") + 1])
output_json

{'athletes': [{'name': 'Michael Jordan', 'sport': 'Basketball'},
  {'name': 'Babe Ruth', 'sport': 'Baseball'},
  {'name': 'Muhammad Ali', 'sport': 'Boxing'},
  {'name': 'Serena Williams', 'sport': 'Tennis'},
  {'name': 'Wayne Gretzky', 'sport': 'Hockey'},
  {'name': 'Michael Phelps', 'sport': 'Swimming'},
  {'name': 'Usain Bolt', 'sport': 'Track and Field'},
  {'name': 'Mia Hamm', 'sport': 'Soccer'},
  {'name': 'Michael Schumacher', 'sport': 'Formula 1 Racing'},
  {'name': 'Simone Biles', 'sport': 'Gymnastics'}]}

매우 길고 복잡한 프롬프트에서 여러 JSON 객체를 출력해야 하는 경우, 단순히 "{" 와 "}" 문자를 검색하는 것만으로는 부족할 수 있습니다. 이러한 경우, Claude가 각 JSON 객체를 특정 XML 태그로 감싸도록 지시하여 나중에 손쉽게 추출할 수 있도록 하는 방법이 유용합니다.

In [13]:
message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[
        {
            "role": "user", 
            "content": """유명 운동선수 5명의 이름과 각 선수의 스포츠 종목을 포함하는 JSON 딕셔너리를 만들어 주십시오.
이 딕셔너리는 <athlete_sports> XML 태그로 감싸주십시오. 

그런 다음, 각 운동선수에 대해 별도의 JSON 딕셔너리를 추가로 출력해주십시오. 이 추가 딕셔너리 각각에는 다음 내용이 포함되어야 합니다:
- 운동선수의 이름(first name)과 성(last name)을 나타내는 두 개의 키를 포함합니다.
- 각 키의 값으로는 해당 이름(또는 성)과 동일한 첫 글자로 시작하는 세 개의 영단어를 나열합니다.
이 추가 딕셔너리 각각은 별도의 <athlete_name> XML 태그로 감싸주십시오."""
        },
        {
            "role": "assistant",
            "content": "요청하신 JSON 데이터는 다음과 같습니다:"
        }
    ],
).content[0].text
print(message)

 

<athlete_sports>
{
  "Michael Jordan": "Basketball",
  "Serena Williams": "Tennis",
  "Lionel Messi": "Soccer", 
  "Usain Bolt": "Track and Field",
  "Michael Phelps": "Swimming"
}
</athlete_sports>

<athlete_name>
{
  "first": ["Magnificent", "Motivating", "Memorable"],
  "last": ["Joyful", "Jumping", "Jocular"]
}
</athlete_name>

<athlete_name>
{
  "first": ["Skillful", "Strong", "Superstar"],
  "last": ["Winning", "Willful", "Wise"]
}
</athlete_name>

<athlete_name>
{
  "first": ["Legendary", "Lively", "Leaping"],
  "last": ["Magical", "Marvelous", "Masterful"]  
}
</athlete_name>

<athlete_name>
{
  "first": ["Unbeatable", "Unbelievable", "Unstoppable"],
  "last": ["Brave", "Bold", "Brilliant"]
}
</athlete_name>

<athlete_name>
{
  "first": ["Marvelous", "Methodical", "Medalist"],
  "last": ["Powerful", "Persevering", "Precise"]
}
</athlete_name>


이제 정규 표현식을 사용하여 모든 딕셔너리들을 추출할 수 있습니다.

In [14]:
import re
def extract_between_tags(tag: str, string: str, strip: bool = False) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list

athlete_sports_dict = json.loads(extract_between_tags("athlete_sports", message)[0])
athlete_name_dicts = [
    json.loads(d)
    for d in extract_between_tags("athlete_name", message)
]

In [15]:
pprint(athlete_sports_dict)

{'Lionel Messi': 'Soccer',
 'Michael Jordan': 'Basketball',
 'Michael Phelps': 'Swimming',
 'Serena Williams': 'Tennis',
 'Usain Bolt': 'Track and Field'}


In [16]:
pprint(athlete_name_dicts, width=1)

[{'first': ['Magnificent',
            'Motivating',
            'Memorable'],
  'last': ['Joyful',
           'Jumping',
           'Jocular']},
 {'first': ['Skillful',
            'Strong',
            'Superstar'],
  'last': ['Winning',
           'Willful',
           'Wise']},
 {'first': ['Legendary',
            'Lively',
            'Leaping'],
  'last': ['Magical',
           'Marvelous',
           'Masterful']},
 {'first': ['Unbeatable',
            'Unbelievable',
            'Unstoppable'],
  'last': ['Brave',
           'Bold',
           'Brilliant']},
 {'first': ['Marvelous',
            'Methodical',
            'Medalist'],
  'last': ['Powerful',
           'Persevering',
           'Precise']}]


요약하자면 다음과 같습니다:

- 문자열 파싱을 사용하여 "```json"과 "```" 사이의 텍스트를 추출함으로써 JSON을 얻을 수 있습니다.
- 어시스턴트 메시지의 일부를 미리 채워 넣음으로써 JSON *앞부분*의 서론을 제거할 수 있습니다. (하지만 이 방법은 Claude가 JSON 출력을 시작하기 전에 더 높은 수준의 추론을 위해 "연쇄적 사고(Chain of Thought)"를 수행할 가능성을 배제합니다.)
- 중단 시퀀스(stop sequence)를 사용하여 JSON *뒷부분*에 오는 불필요한 텍스트를 제거할 수 있습니다.
- 더 복잡한 프롬프트의 경우, Claude에게 XML 태그 내에 JSON을 출력하도록 지시하여 나중에 손쉽게 데이터를 수집할 수 있도록 할 수 있습니다.